In [339]:
import pandas as pd
import string
from nltk.corpus import stopwords

In [340]:
import nltk

In [341]:
nltk.download ('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shweta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [342]:
leads  = pd.read_excel("1000 leads.xlsx")

In [343]:
leads

,Lead Name,Location,Status,Status information
0,Raja,hyderabad,Not Converted,"14/8/prema: share me details, available in evn..."
1,Anirudh Reddy,pune,Not Converted,"14/8/prema: cal me tmrw, shared details to ema..."
2,Sapna Dewani,bangalore,Converted,16|AuG|moHan:rnr
3,suresh,mumbai,Not Converted,14/8/17(Surendra):i want only Server 16|AuG|mo...
4,Akshay Shinde,hyderabad,Not Converted,"14/8/prema:rnr 16/8/prema: gave info, he said ..."
...,...,...,...,...
996,vipin,bangalore,Not Converted,25/4/17(Surendra):please send me details tomor...
997,dheeraj,chennai,Not Converted,"24/4/17(prema): need CT, but our venue is far...."
998,kuldeep singh,bangalore,Not Converted,24/Apr/moHan:intrstd in evng batch trail perd ...
999,ankur sharma,bangalore,Not Converted,1/5/17(Surendra):please share me details 11/5/...


In [344]:
leads ['Status '].replace ('Conveted','Converted',inplace=True)
leads ['Status '].replace ('NOt Converted','Not Converted',inplace=True)
leads ['Status '].replace ('Converted ','Converted',inplace=True)
leads ['Status '].replace ('Zero','No information',inplace=True)

In [345]:
leads ['Status '].unique()

array(['Not Converted', 'Converted', nan], dtype=object)

In [346]:
leads ['Status '].isnull().sum()

3

In [347]:
leads['Status '].fillna('not mentioned',inplace=True)

In [348]:
leads['Status information'].isnull().sum()

24

In [349]:
leads['Status information'].fillna('not mentioned',inplace=True)

In [350]:
leads.groupby('Status ').describe()

Lead Name                          Location                    \
                  count unique          top freq    count unique        top   
Status                                                                        
Converted           131    128       Rashmi    2      128     19  bangalore   
Not Converted       867    765       Suresh    5      849     51  bangalore   
not mentioned         3      3  Vaijayanthi    1        3      3  hyderabad   

                   Status information         \
              freq              count unique   
Status                                         
Converted       45                131    130   
Not Converted  293                867    841   
not mentioned    1                  3      3   

                                                                       
                                                             top freq  
Status                                                                 
Converted                                          not mentioned    2  
Not Converted                                      not mentioned   22  
not mentioned  17/JulY/moHan:need cls room 20/JulY/moHan:not now    1

In [351]:
def message_text_process(mess):
    no_punctuation = [char for char in mess if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return [word for word in no_punctuation.split() if word.lower() 
            not in stopwords.words('english')]

In [353]:
leads['Status information'].head(5).apply(message_text_process)

0    [148prema, share, details, available, evng, 18...
1    [148prema, cal, tmrw, shared, details, email, ...
2                                      [16AuGmoHanrnr]
3    [14817Surendrai, want, Server, 16AuGmoHancal, ...
4    [148premarnr, 168prema, gave, info, said, reve...
Name: Status information, dtype: object

In [354]:
from sklearn.feature_extraction.text import CountVectorizer #DTM

In [355]:
bag_words = CountVectorizer(analyzer = message_text_process).fit(leads['Status information'])

In [356]:
message_bagwords = bag_words.transform(leads['Status information'])

In [357]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(message_bagwords)

In [210]:
tfidf_transformer

TfidfTransformer()

In [211]:
bag_words

CountVectorizer(analyzer=<function message_text_process at 0x0000020327F69C10>)

In [358]:
message_tfidf = tfidf_transformer.transform(message_bagwords)

In [324]:
print(message_tfidf.shape)
print(message_tfidf)

(1001, 3184)
  (0, 3074)	0.2584369209100249
  (0, 2968)	0.1845545751797856
  (0, 2829)	0.3205360129959093
  (0, 2809)	0.3793645463964475
  (0, 2573)	0.1939457005797172
  (0, 2454)	0.24125452603846578
  (0, 2364)	0.12637893920071414
  (0, 2333)	0.3205360129959093
  (0, 2184)	0.2683172116901641
  (0, 1906)	0.39609437948831866
  (0, 609)	0.3260786712036078
  (0, 361)	0.3205360129959093
  (1, 3052)	0.1552216677595328
  (1, 2969)	0.1031596387293627
  (1, 2968)	0.1366187113801645
  (1, 2918)	0.1649734814716771
  (1, 2885)	0.29321355841709357
  (1, 2713)	0.17568823481685542
  (1, 2643)	0.18064186706226892
  (1, 2553)	0.19862499570394887
  (1, 2425)	0.5587216154917154
  (1, 2364)	0.18710701484776682
  (1, 2251)	0.12966019504258214
  (1, 2244)	0.2808291012103831
  (1, 2228)	0.1582333227419244
  :	:
  (1000, 2574)	0.1858783652405565
  (1000, 2553)	0.1314681752647276
  (1000, 2505)	0.10643490459555219
  (1000, 2364)	0.12384452286121383
  (1000, 2230)	0.07837793576593804
  (1000, 2228)	0.209466549

In [359]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(message_tfidf,leads['Status '] , test_size = 0.2 , random_state=1)

In [360]:
leads.head ()

,Lead Name,Location,Status,Status information
0,Raja,hyderabad,Not Converted,"14/8/prema: share me details, available in evn..."
1,Anirudh Reddy,pune,Not Converted,"14/8/prema: cal me tmrw, shared details to ema..."
2,Sapna Dewani,bangalore,Converted,16|AuG|moHan:rnr
3,suresh,mumbai,Not Converted,14/8/17(Surendra):i want only Server 16|AuG|mo...
4,Akshay Shinde,hyderabad,Not Converted,"14/8/prema:rnr 16/8/prema: gave info, he said ..."


In [89]:
# from sklearn.naive_bayes import MultinomialNB
# leads_status = MultinomialNB().fit(X_train,y_train)

In [361]:
from sklearn import svm
leads_status = svm.SVC(kernel='linear', C=5,gamma= 10).fit(X_train, y_train)

In [362]:
predicted = leads_status.predict(X_test)

In [363]:
predicted

array(['Not Converted', 'Not Converted', 'Not Converted', 'Not Converted',
       'Not Converted', 'Not Converted', 'Not Converted', 'Not Converted',
       'Not Converted', 'Not Converted', 'Not Converted', 'Not Converted',
       'Not Converted', 'Not Converted', 'Not Converted', 'Not Converted',
       'Not Converted', 'Not Converted', 'Not Converted', 'Not Converted',
       'Not Converted', 'Not Converted', 'Not Converted', 'Not Converted',
       'Not Converted', 'Not Converted', 'Not Converted', 'Not Converted',
       'Not Converted', 'Not Converted', 'Not Converted', 'Not Converted',
       'Not Converted', 'Not Converted', 'Not Converted', 'Not Converted',
       'Not Converted', 'Not Converted', 'Not Converted', 'Not Converted',
       'Not Converted', 'Not Converted', 'Not Converted', 'Not Converted',
       'Not Converted', 'Not Converted', 'Not Converted', 'Not Converted',
       'Not Converted', 'Not Converted', 'Not Converted', 'Not Converted',
       'Not Converted', '

In [364]:
expected = y_test

from sklearn import metrics
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

               precision    recall  f1-score   support

    Converted       0.43      0.12      0.19        25
Not Converted       0.88      0.98      0.92       174
not mentioned       0.00      0.00      0.00         2

     accuracy                           0.86       201
    macro avg       0.43      0.37      0.37       201
 weighted avg       0.81      0.86      0.82       201

[[  3  22   0]
 [  4 170   0]
 [  0   2   0]]


C:\Users\Shweta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Shweta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Shweta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [365]:
from sklearn.metrics import accuracy_score

accuracy_score(expected,predicted)

0.8606965174129353

In [366]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

RandomForestClassifier()

In [44]:
# from sklearn.svm import SVC
# classifier = SVCClassifier()
# classifier.fit(X_train, y_train)

In [367]:
y_pred = classifier.predict(X_test)

In [368]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, y_pred)

In [369]:
score

0.8656716417910447

In [370]:
# pickling the model
import pickle
pickle_out = open("classifier.pkl", "wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()

In [371]:
leads ['Status ']

0       Not Converted
1       Not Converted
2           Converted
3       Not Converted
4       Not Converted
            ...      
996     Not Converted
997     Not Converted
998     Not Converted
999     Not Converted
1000    Not Converted
Name: Status , Length: 1001, dtype: object